In [190]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from unidecode import unidecode
from pathlib import Path
from datetime import datetime, timedelta
from lingua import Language, LanguageDetectorBuilder
import pandas as pd
import json
import time
import random

In [ ]:
['LUX Saint Gilles', 'LUX Belle Mare', 'LUX Grand Gaube', 'Chalet Iona', 'Les Rives marines', 'Hôtel du Golfe Ajaccio', 'Dolce Notte', 'Restaurant Chacouette', 'Hotel Ibiza', 'ESF Chamonix', 'La Fine Bouche', 'Le Comptoir des Alpes', 'Pierre & Vacances Residence La Rivière', 'Grand Hôtel des Alpes', 'Le Lido', 'Office de Tourisme de Chamonix-Mont-Blanc', 'Les Chalets du Berger', 'Le Château de Candie', 'MV Transport']

In [191]:
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.70',
    'Mozilla/5.0 (Linux; Android 11; SM-G991U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.6 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Windows NT 7.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Version/11.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; Pixel 3 XL Build/QP1A.190711.020) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 OPR/79.0.4143.66'
    ]
        
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-geolocation")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-fingerprinting')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--disable-gpu')
# chrome_options.add_experimental_option('excludeSwitch',['enable-logging']) 
chrome_options.add_argument('--log-level=3') 
chrome_options.add_argument(f"user-agent={random.choices(user_agents)}")
chrome_options.add_argument('--incognito')

In [192]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('disable-translate')
chrome_options.add_argument('--lang=fr')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_extension("./canvas_blocker.crx")

In [193]:
url = "https://www.tripadvisor.fr/Hotel_Review-g488104-d316747-Reviews-LUX_Grand_Gaube-Grand_Gaube.html"

In [194]:
driver = webdriver.Chrome(options=chrome_options)

In [195]:
driver.get("https://google.com")

In [196]:
search_input = driver.find_element(By.XPATH, "//textarea[@title='Rechercher']")

In [197]:
search_input.click()
time.sleep(1)
search_key = "LUX Grand Gaube".split(' ')
for key in search_key:
    search_input.send_keys(f"{key} ")
    time.sleep(.5)
search_input.send_keys(Keys.RETURN)

In [198]:
time.sleep(5)

In [199]:
links = driver.find_elements(By.CSS_SELECTOR, "div.srKDX.cvP2Ce")

In [200]:
len(links)

9

In [201]:
def search_link(elements:object, netloc:str, search:str):
    for i in range(len(elements)):
        tag_a = BeautifulSoup(elements[i].get_attribute('innerHTML'), 'lxml').find_all('a', href=True)
        for a in tag_a:
            try:
                if netloc in a['href']:
                    find = 0
                    for item in search.split(' '):
                        if item.lower() in a['href'].lower():
                            find += 1
                    print(f"{int(find / len(search.split(' '))) * 100 } %")
                    if (find / len(search.split(' '))) >= 0.6:
                        print(a['href'])
                        return a['href'], i
            except:
                pass

In [202]:
find_link, index = search_link(links, 'tripadvisor', "LUX Grand Gaube")

100 %
https://www.tripadvisor.fr/Hotel_Review-g488104-d316747-Reviews-LUX_Grand_Gaube-Grand_Gaube.html


In [203]:
if find_link == url:
    links[index].click()